In [1]:
import fitz
from operator import itemgetter
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles

In [4]:
pdf_path = "C:/Users/wloon/Desktop/NTU/Hackathon/research1.pdf"
pdf_doc = fitz.open(pdf_path)
fonts(pdf_doc)

([('9.800000190734863', 645),
  ('7.5', 246),
  ('8.0', 236),
  ('10.0', 55),
  ('9.803299903869629', 34),
  ('7.0', 32),
  ('7.450399875640869', 29),
  ('5.25', 19),
  ('11.0', 18),
  ('9.199999809265137', 16),
  ('7.699999809265137', 14),
  ('6.860000133514404', 13),
  ('10.300000190734863', 12),
  ('4.900000095367432', 11),
  ('9.0', 10),
  ('5.599999904632568', 6),
  ('6.300000190734863', 5),
  ('24.0', 4),
  ('13.0', 2),
  ('12.430000305175781', 1)],
 {'8.0': {'size': 8.0, 'font': 'MyriadPro-Regular3'},
  '13.0': {'size': 13.0, 'font': 'MyriadPro-Bold'},
  '7.0': {'size': 7.0, 'font': 'MyriadPro-Regular'},
  '12.430000305175781': {'size': 12.430000305175781,
   'font': 'MyriadPro-Regular2'},
  '24.0': {'size': 24.0, 'font': 'MyriadPro-Regular'},
  '11.0': {'size': 11.0, 'font': 'MyriadPro-Light'},
  '7.699999809265137': {'size': 7.699999809265137, 'font': 'MyriadPro-Light'},
  '10.300000190734863': {'size': 10.300000190734863, 'font': 'MyriadPro-Bold'},
  '10.0': {'size': 10.0, 'f

In [5]:
def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.
    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict
    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag 
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag

In [6]:
font_counts, styles = fonts(pdf_doc, granularity=False)
size_tag = font_tags(font_counts, styles)

In [7]:
def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                header_para.append(block_string)

    return header_para

In [8]:
for line in headers_para(pdf_doc, size_tag):
    print(line)

<s3>Lamptey  et al. BMC Health Services Research          (2023) 23:199  | https://doi.org/10.1186/s12913-023-09188-y|

<h2>RESEARCH|

<s7>© The Author(s) 2023.  Open Access  This article is licensed under a Creative Commons Attribution 4.0 International License, which | permits use, sharing, adaptation, distribution and reproduction in any medium or format, as long as you give appropriate credit to the | original author(s) and the source, provide a link to the Creative Commons licence, and indicate if changes were made. The images or | other third party material in this article are included in the article’s Creative Commons licence, unless indicated otherwise in a credit line | to the material. If material is not included in the article’s Creative Commons licence and your intended use is not permitted by statutory | regulation or exceeds the permitted use, you will need to obtain permission directly from the copyright holder. To view a copy of this | licence, visit  http:// creat ivec